In [174]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

C:\ProgramData\Anaconda3\envs\Brain_AI\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [175]:
from utils.opv import OpvModel
mymodel2=OpvModel("face-detection-adas-0001",device="CPU",fp="FP32")

C:\ProgramData\Anaconda3\envs\Brain_AI\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
C:\Intel_JS\openVINO\utils\opv.py:61: DeprecationWarning: Reading network using constructor is deprecated. Please, use IECore.read_network() method instead
  net = IENetwork(model=model_xml, weights=model_bin)
C:\Intel_JS\openVINO\utils\opv.py:65: DeprecationWarning: IEPlugin class is deprecated. Please use IECore class instead.
  plugin = IEPlugin(device=device)


In [176]:
image = cv2.imread('images/friends.JPEG')
predictions=mymodel2.Predict(image)

C:\ProgramData\Anaconda3\envs\Brain_AI\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [177]:
mymodel1=OpvModel("age-gender-recognition-retail-0013",device="CPU",fp="FP32")

C:\ProgramData\Anaconda3\envs\Brain_AI\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
C:\Intel_JS\openVINO\utils\opv.py:61: DeprecationWarning: Reading network using constructor is deprecated. Please, use IECore.read_network() method instead
  net = IENetwork(model=model_xml, weights=model_bin)
C:\Intel_JS\openVINO\utils\opv.py:65: DeprecationWarning: IEPlugin class is deprecated. Please use IECore class instead.
  plugin = IEPlugin(device=device)


In [211]:
def RecogBoxes(age):
    gender_dic = {0: 'female', 1: 'male'}
    #gender_recog = gender_dic[np.argmax(gender[0,:,0,0])]
    age_recog = round(age[0][0][0][0] * 100, 1)
    print(age_recog)
    print(age)
    return age_recog

C:\ProgramData\Anaconda3\envs\Brain_AI\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [212]:
def DrawBoundingBoxes(predictions, image, conf=0.8):
    mymodel1=OpvModel("age-gender-recognition-retail-0013",device="CPU",fp="FP32")
    
    canvas=image.copy()
    predictions_1=predictions[0][0] # numpy array 타입
    confidence=predictions_1[:,2]   # 2번 데이터만
    topresults=predictions_1[(confidence>conf)]  # conf 보다 큰 데이터만
    (h,w)=canvas.shape[:2]
    for detection in topresults:
        box=detection[3:7]*np.array([w,h,w,h])
        (xmin,ymin,xmax,ymax)=box.astype("int")
        img=canvas[ymin:ymax,xmin:xmax].copy()
        
        predictions_face_age = mymodel1.Predict(img)
        age=RecogBoxes(predictions_face_age)
        cv2.rectangle(canvas,(xmin,ymin),(xmax,ymax),(0,0,255),4)
        #cv2.putText(canvas,emo+" "+str(round(max_emo_val*100,1))+"%",(xmin,ymin),cv2.FONT_HERSHEY_SIMPLEX,0.6,(255,0,0),2)
    #cv2.putText(canvas,str(len(topresults))+" face(s) detected",(50,50),cv2.FONT_HERSHEY_SIMPLEX,0.6,(255,0,0),2)
    
    return canvas

C:\ProgramData\Anaconda3\envs\Brain_AI\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [213]:
cv2.imshow("canvas",DrawBoundingBoxes(predictions,image))
cv2.waitKey(0)
cv2.destroyAllWindows()

C:\ProgramData\Anaconda3\envs\Brain_AI\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
C:\Intel_JS\openVINO\utils\opv.py:61: DeprecationWarning: Reading network using constructor is deprecated. Please, use IECore.read_network() method instead
  net = IENetwork(model=model_xml, weights=model_bin)
C:\Intel_JS\openVINO\utils\opv.py:65: DeprecationWarning: IEPlugin class is deprecated. Please use IECore class instead.
  plugin = IEPlugin(device=device)


37.1
[[[[0.37071723]]]]
23.5
[[[[0.23486753]]]]
21.1
[[[[0.21095943]]]]
24.8
[[[[0.24844998]]]]
